<a href="https://www.kaggle.com/code/rawanmoamed/eeg-classification-ccnn?scriptVersionId=285440418" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install torch_scatter torcheeg torch_geometric -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.4/251.4 kB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.1/295.1 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 63.3 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.utils as utils
from torch.utils.data import DataLoader, Subset,WeightedRandomSampler
from torcheeg.models import CCNN
from torcheeg import transforms
from torcheeg.transforms import ToGrid
from torcheeg.datasets import SEEDIVDataset,SEEDIVFeatureDataset
from torcheeg.datasets.constants import SEED_IV_CHANNEL_LOCATION_DICT
from torcheeg.transforms import ToG
from torcheeg.datasets.constants import SEED_IV_ADJACENCY_MATRIX
from torcheeg.models import DGCNN
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import ReduceLROnPlateau 
# --- THE MAIN SUBJECT LOOP ---

import torch_geometric.loader as geom_loader # Special loader for graphs
import copy
import scipy.signal as signal
import random
import numpy as np

In [3]:
import shutil
import os

if os.path.exists('./tmp_out/seed_iv_features'):
    shutil.rmtree('./tmp_out/seed_iv_features')
    print("Old cache deleted. Data will be re-processed.")

In [4]:
# 1. Setup Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
from torcheeg.datasets.constants import SEED_IV_CHANNEL_LOCATION_DICT
from torcheeg.transforms import ToGrid

dataset = SEEDIVFeatureDataset(
    io_path='./tmp_out/seed_iv_features',
    root_path='/kaggle/input/seed-iv/eeg_feature_smooth',
    feature=['de_LDS'],
    num_worker=0,
    offline_transform=transforms.Compose([
        ToGrid(SEED_IV_CHANNEL_LOCATION_DICT),
        transforms.MinMaxNormalize(),
        transforms.Lambda(lambda x: torch.tensor(x).float())
        
    ]),
    label_transform=transforms.Compose([
        transforms.Select('emotion')
    ])
)


[2025-12-11 14:00:12] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./tmp_out/seed_iv_features.
[2025-12-11 14:00:12] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|          | 0/45 [00:00<?, ?it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 0it [00:00, ?it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 3it [00:00, 29.67it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 23it [00:00, 127.96it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 42it [00:00, 152.38it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 63it [00:00, 173.82it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 84it [00:00, 185.80it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_201511

In [6]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42) 

In [7]:
subjects = sorted(dataset.info['subject_id'].unique()) # Get list of all 15 subjects

In [8]:
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

In [9]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in loader:
        x = batch[0].to(device)
        y = batch[1].to(device)


        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = out.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

    return total_loss / len(loader), correct / total

In [10]:
def evaluate(model, loader, criterion):
    model.eval()
    
    total_loss = 0.0  
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in loader:
            x = batch[0].to(device)
            y = batch[1].to(device)

            out = model(x)
            
            loss = criterion(out, y)
            total_loss += loss.item()

            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return total_loss / len(loader), correct / total

In [11]:
all_subject_accuracies = []

for subject_id in subjects:
    print(f"\nProcessing SUBJECT: {subject_id}")
    
    # --- Data Split by Trials ---
    sub_df = dataset.info[dataset.info['subject_id'] == subject_id]
    all_trials = list(range(1, 25))
    random.seed(42)
    test_trials = random.sample(all_trials, 5)
    train_trials = [t for t in all_trials if t not in test_trials]

    train_indices = sub_df[sub_df['trial_id'].isin(train_trials)].index.tolist()
    test_indices = sub_df[sub_df['trial_id'].isin(test_trials)].index.tolist()

    train_set = Subset(dataset, train_indices)
    test_set = Subset(dataset, test_indices)

    # --- Sampler for Class Imbalance ---
    y_train_indices = train_set.indices
    raw_labels = dataset.info.iloc[y_train_indices]['emotion'].values
    class_counts = np.bincount(raw_labels)
    class_weights = 1. / class_counts
    sample_weights = [class_weights[y] for y in raw_labels]

    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

    train_loader = DataLoader(train_set, batch_size=64, sampler=sampler, num_workers=0)
    test_loader = DataLoader(test_set, batch_size=64, shuffle=False, num_workers=0)

    # --- Model ---
    model = CCNN(
        num_classes=4,
        in_channels=5,        
        grid_size=(9, 9),     
        dropout=0.1
    ).to(device)
    model.apply(weights_init)

    # --- Criterion & Optimizer ---
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = CosineAnnealingLR(optimizer, T_max=100)

    epochs = 100
    best_acc = 0.0

    # --- Training Loop ---
    for epoch in range(epochs):
        model.train()
        train_loss_sum = 0
        correct = 0
        total = 0

        for batch in train_loader:
            x, y = batch
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

            train_loss_sum += loss.item()
            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

        avg_train_loss = train_loss_sum / len(train_loader)
        train_acc = correct / total

        val_loss, val_acc = evaluate(model, test_loader, criterion)
        scheduler.step()

        is_best = False
        if val_acc > best_acc:
            best_acc = val_acc
            is_best = True

        if is_best or (epoch + 1) % 10 == 0:
            status = "(*) BEST" if is_best else ""
            print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f} | Train Acc: {train_acc:.3f} | Val Acc: {val_acc:.3f} {status}")

    all_subject_accuracies.append(best_acc)
    print(f"Subject {subject_id} FINAL BEST ACC: {best_acc*100:.2f}%")

# --- Final Summary ---
print("\n" + "="*50)
print("FINAL RESULTS SUMMARY")
print("="*50)
final_avg_acc = np.mean(all_subject_accuracies)
final_std_acc = np.std(all_subject_accuracies)
print(f"Subjects: {len(subjects)} | Average Accuracy: {final_avg_acc*100:.2f}% | Std: {final_std_acc*100:.2f}")
print(f"Accuracies List: {[round(x*100,2) for x in all_subject_accuracies]}")


Processing SUBJECT: 1


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch 1/100 | Train Loss: 6.4076 | Val Loss: 1.3122 | Train Acc: 0.249 | Val Acc: 0.382 (*) BEST
Epoch 4/100 | Train Loss: 1.4108 | Val Loss: 1.3186 | Train Acc: 0.340 | Val Acc: 0.425 (*) BEST
Epoch 10/100 | Train Loss: 1.2047 | Val Loss: 1.4381 | Train Acc: 0.445 | Val Acc: 0.279 
Epoch 11/100 | Train Loss: 1.1195 | Val Loss: 1.3582 | Train Acc: 0.531 | Val Acc: 0.487 (*) BEST
Epoch 20/100 | Train Loss: 1.0412 | Val Loss: 1.4605 | Train Acc: 0.556 | Val Acc: 0.430 
Epoch 24/100 | Train Loss: 0.8915 | Val Loss: 1.2909 | Train Acc: 0.682 | Val Acc: 0.546 (*) BEST
Epoch 30/100 | Train Loss: 0.8012 | Val Loss: 1.3884 | Train Acc: 0.711 | Val Acc: 0.414 
Epoch 40/100 | Train Loss: 0.7028 | Val Loss: 1.0467 | Train Acc: 0.738 | Val Acc: 0.507 
Epoch 50/100 | Train Loss: 0.5035 | Val Loss: 1.6109 | Train Acc: 0.860 | Val Acc: 0.406 
Epoch 53/100 | Train Loss: 0.5193 | Val Loss: 1.0556 | Train Acc: 0.844 | Val Acc: 0.619 (*) BEST
Epoch 60/100 | Train Loss: 0.4319 | Val Loss: 1.1954 | Train A